<a href="https://colab.research.google.com/github/davidcechak/2015s2f1/blob/master/colab_ready_playground_with_preprocessed_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install torchmetrics

  Using cached torchmetrics-1.2.0-py3-none-any.whl (805 kB)
  Using cached lightning_utilities-0.9.0-py3-none-any.whl (23 kB)


In [2]:
# Modules will be reloaded every time a cell is executed
%load_ext autoreload
%autoreload 2

In [1]:
!git clone https://github.com/ML-Bioinfo-CEITEC/miRNA.git

Cloning into 'miRNA'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 129 (delta 51), reused 84 (delta 25), pack-reused 0
Receiving objects: 100% (129/129), 30.02 MiB | 20.59 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [ ]:
%cd miRNA

/content/miRNA


### Model training

In [3]:
DATASET_NAME = 'mirna_fcs'
MIRNA_NAMES = [
  'hsa-miR-106b-5p',
  'hsa-let-7c-5p',
  'hsa-miR-16-5p',
]
DATA_DIR = 'data'

RANDOM_STATE = 42
BATCH_SIZE = 32

In [5]:
import pandas as pd

COLAB_PATH = 'miRNA/'

input_dataset_file_path_train = COLAB_PATH + 'data/{}.{}.train.positive_fc_to_zero.signals.compressed_zeros.remove_genes_without_signal.zero_signal_to_negative.normalize.pkl'.format(
    DATASET_NAME,
    MIRNA_NAMES[0]
)
df_train = pd.read_pickle(input_dataset_file_path_train)

input_dataset_file_path_test = COLAB_PATH + 'data/{}.{}.test.positive_fc_to_zero.signals.compressed_zeros.predict_genes_without_signal.zero_signal_to_negative.normalize.fill_empty_signal.pkl'.format(
    DATASET_NAME,
    MIRNA_NAMES[0]
)
df_test = pd.read_pickle(input_dataset_file_path_test)

for name in MIRNA_NAMES[1:]:
    input_dataset_file_path_train = COLAB_PATH + 'data/{}.{}.train.positive_fc_to_zero.signals.compressed_zeros.remove_genes_without_signal.zero_signal_to_negative.normalize.pkl'.format(
        DATASET_NAME,
        name
    )
    df_train = pd.concat(
        [df_train, pd.read_pickle(input_dataset_file_path_train)],
        axis='index',
        ignore_index=True,
    )

    input_dataset_file_path_test = COLAB_PATH + 'data/{}.{}.test.positive_fc_to_zero.signals.compressed_zeros.predict_genes_without_signal.zero_signal_to_negative.normalize.fill_empty_signal.pkl'.format(
        DATASET_NAME,
        name
    )
    df_test = pd.concat(
        [df_test, pd.read_pickle(input_dataset_file_path_test)],
        axis='index',
        ignore_index=True,
    )

If you want to pad the signals to the length of the longest sequence

In [6]:
def pad_2d_array_to_longest_subarray_length(arr, pad_value=0):
    # Find the maximum length of sublists
    max_len = max(len(sublist) for sublist in arr)

    # Pad the sublists to max length
    padded_arr = [sublist + [pad_value] * (max_len - len(sublist)) for sublist in arr]

    return padded_arr

In [7]:
df_train['signal'] = pad_2d_array_to_longest_subarray_length(df_train['signal'])
df_test['signal'] = pad_2d_array_to_longest_subarray_length(df_test['signal'])

In [8]:
from sklearn.model_selection import train_test_split

#split validation set
df_train, df_val = train_test_split(
    df_train,
    test_size = 0.1,
    random_state = RANDOM_STATE,
    shuffle=True
)

In [9]:
print(
    df_train['gene_symbol'].size,
    df_test['gene_symbol'].size,
    df_val['gene_symbol'].size,
)

10186 6101 1132


If you need pytorch dataloader

In [ ]:
from dataset import get_train_dataloader, get_val_dataloader, get_test_dataloader
import torch

train_loader = get_train_dataloader([torch.tensor(x) for x in df_train['signal']], df_train['fold_change'].values, BATCH_SIZE)
val_loader = get_val_dataloader([torch.tensor(x) for x in df_val['signal']], df_val['fold_change'].values, BATCH_SIZE)
test_loader = get_test_dataloader([torch.tensor(x) for x in df_test['signal']], df_test['fold_change'].values, BATCH_SIZE)